In [ ]:
! pip install sqlalchemy psycopg2 pandas


In [5]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

USER = "postgres"
PASSWORD = "ycode"
HOST = "localhost"
PORT = "5432"
DB_NAME = "breif5_football_db"

# Encoder le mot de passe pour gérer les caractères spéciaux
password_encoded = quote_plus(PASSWORD)
user_encoded = quote_plus(USER)

DATABASE_URL = f"postgresql+psycopg2://{user_encoded}:{password_encoded}@{HOST}:{PORT}/{DB_NAME}"

engine = create_engine(DATABASE_URL, echo=True)

try:
    with engine.connect() as conn:
        print("✅ Connexion réussie à la base PostgreSQL !")
except Exception as e:
    print("❌ Erreur de connexion :", e)


2025-10-28 20:51:34,920 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-28 20:51:34,921 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-28 20:51:34,922 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-28 20:51:34,922 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-28 20:51:34,923 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-28 20:51:34,924 INFO sqlalchemy.engine.Engine [raw sql] {}
✅ Connexion réussie à la base PostgreSQL !


## Create tables in database


In [8]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey, Enum, MetaData , Date , Numeric

metadata = MetaData()

competition = Table(
    "competition", metadata,
    Column("idcompetition", Integer, primary_key=True, autoincrement=True),
    Column("nomcompetition", String(100), nullable=False)
)

saison = Table(
    "saison", metadata,
    Column("idsaison", Integer, primary_key=True, autoincrement=True),
    Column("annee", String(20), nullable=False)
)

equipe = Table(
    "equipe", metadata,
    Column("idequipe", Integer, primary_key=True, autoincrement=True),
    Column("nomequipe", String(100), nullable=False),
)

joueur = Table(
    "joueur", metadata,
    Column("idjoueur", Integer, primary_key=True, autoincrement=True),
    Column("nomjoueur", String(100), nullable=False),
    Column("position", String(10)),
    Column("nationalite", String(50)),
    Column("idequipe", Integer, ForeignKey("equipe.idequipe"))
)

matchs = Table(
    "match", metadata,
    Column("idmatch", Integer, primary_key=True, autoincrement=True),
    Column("date_match", Date()),
    Column("score", String(10)),
    Column("round" , String(50)) ,
    Column("jour"  , String(10)) ,
    Column("arbitre" , String(100)),
    Column("idcompetition" , Integer , ForeignKey("competition.idcompetition")) , 
    Column("idSaision" , Integer , ForeignKey("saison.idsaison"))
    )
resultatmatch = Table(
    "resultatmatch", metadata,
    Column("idresultat", Integer, primary_key=True, autoincrement=True),
    Column("idmatch", Integer, ForeignKey("match.idmatch")),
    Column("idequipe", Integer, ForeignKey("equipe.idequipe")),
    Column("venue", Enum("Home", "Away", "Neutral", name="venue_enum")),
    Column("butsmarques", Integer),
    Column("butsconcedes", Integer),
    Column("resultat", Enum("Victoire", "Défaite", "Nul", name="resultat_enum")),
    Column("xG", Numeric(3, 1)),
    Column("xGA", Numeric(3, 1)),
    Column("possession", Numeric(4, 1)),
    Column("capitaine", String(100)),
    Column("formation", String(20)),
)


statistiquejoueur_saison = Table(
    "statistiquejoueur_saison", metadata,
    Column("idstats", Integer, primary_key=True, autoincrement=True),
    Column("idjoueur", Integer, ForeignKey("joueur.idjoueur")),
    Column("matchs_joues", Integer),
    Column("matchs_titulaire", Integer),
    Column("minutes_jouees", Integer),
    Column("matchs_90", Numeric(4, 1)),
    Column("buts", Integer),
    Column("passes_decisives", Integer),
    Column("buts_passes", Integer),
    Column("buts_hors_penalty", Integer),
    Column("penalties_marques", Integer),
    Column("penalties_tentes", Integer),
    Column("cartons_jaunes", Integer),
    Column("cartons_rouges", Integer)
)


# Création des tables dans la base
metadata.create_all(engine)
print("✅ Toutes les tables ont été créées avec succès !")


2025-10-29 10:10:41,293 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-29 10:10:41,294 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-10-29 10:10:41,295 INFO sqlalchemy.engine.Engine [cached since 120.2s ago] {'table_name': 'competition', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-10-29 10:10:41,298 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalo

## Insert Data  CSV in tables dataBase

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Configuration des chemins
BRONZE_PATH = "../data/bronze"
SILVER_PATH = "../data/silver"